In [30]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix

## Funciones para el procesamiento

In [31]:
# Función para calcular la ganancia de la empresa
def calcular_ganancia(y_true, y_pred, monto):
    """
    Calcula la ganancia basada en las predicciones:
    - Falso negativo (fraude no detectado) -> pérdida del 100% del monto
    - Transacción aprobada correctamente -> ganancia del 25% del monto
    - Transacción denegada -> sin ganancia
    """
    df_resultado = pd.DataFrame({'fraude_real': y_true, 'fraude_predicho': y_pred, 'monto': monto})

    # Casos donde el fraude real es 1 pero fue predicho como 0 (Falso Negativo)
    perdida = df_resultado[(df_resultado['fraude_real'] == 1) & (df_resultado['fraude_predicho'] == 0)]['monto'].sum()

    # Casos donde se aprueba la transacción (fraude_predicho == 0), se obtiene 25% de ganancia
    ganancia = df_resultado[df_resultado['fraude_predicho'] == 0]['monto'].sum() * 0.25

    # Ganancia total
    ganancia_total = ganancia - perdida

    return ganancia_total


# Función para entrenar y evaluar un modelo con pesos de clase
def train_and_evaluate_model_with_weights(model, X_train, y_train, X_test, y_test, monto_test, model_name):
    # Entrenar el modelo
    model.fit(X_train, y_train)
    
    # Predecir en el conjunto de prueba
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilidades para la clase positiva
    
    # Evaluar el modelo
    print("---------------------------------------------------")
    print(f"\n--- Evaluando modelo: {model_name} ---")
    print("Matriz de confusión:")
    print(confusion_matrix(y_test, y_pred))
    
    print("\nReporte de clasificación:")
    print(classification_report(y_test, y_pred))
    
    print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_proba):.4f}")

    # Calcular la ganancia de la empresa
    ganancia = calcular_ganancia(y_test, y_pred, monto_test)
    print(f"\nGanancia estimada para {model_name}: {ganancia:.2f}")

    return model


# Función para entrenar y evaluar un modelo
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, monto_test, model_name):
    # Entrenar el modelo
    model.fit(X_train, y_train)
    
    # Predecir en el conjunto de prueba
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]  # Probabilidades para la clase positiva
    
    # Evaluar el modelo
    print(f"\n--- Evaluando modelo: {model_name} ---")
    print("Matriz de confusión:")
    print(confusion_matrix(y_test, y_pred))
    
    print("\nReporte de clasificación:")
    print(classification_report(y_test, y_pred))
    
    print(f"AUC-ROC: {roc_auc_score(y_test, y_pred_proba):.4f}")

    # Calcular la ganancia de la empresa
    ganancia = calcular_ganancia(y_test, y_pred, monto_test)
    print(f"\nGanancia estimada para {model_name}: {ganancia:.2f}")

    return model


In [32]:
df_processed = pd.read_csv('../data/df_processed.csv')

X = df_processed.drop(columns=['fraude','monto'])  # Todas las columnas excepto la variable objetivo y monto que se va a usar para el calculo de la ganancia
y = df_processed['fraude']  # Variable objetivo
monto = df_processed['monto'] #se usará solo para la ganancia

# Dividir los datos en conjuntos de entrenamiento y prueba
X_train, X_test, y_train, y_test, monto_train, monto_test = train_test_split(
    X, y, monto, test_size=0.2, random_state=42, stratify=y
)

# Verificar las formas de los conjuntos
print(f"Forma de X_train: {X_train.shape}")
print(f"Forma de X_test: {X_test.shape}")
print(f"Forma de y_train: {y_train.shape}")
print(f"Forma de y_test: {y_test.shape}")

# Calcular el ratio de desbalanceo para XGBoost
scale_pos_weight = (len(y_train) - sum(y_train)) / sum(y_train)

# Definir los modelos con pesos de clase
models = {
    "Regresión Logística": LogisticRegression(random_state=42, class_weight='balanced', max_iter=1000),
    "Random Forest": RandomForestClassifier(random_state=42, class_weight='balanced', n_estimators=100),
    "XGBoost": XGBClassifier(random_state=42, scale_pos_weight=scale_pos_weight, use_label_encoder=False, eval_metric='logloss')
}


Forma de X_train: (120000, 71)
Forma de X_test: (30000, 71)
Forma de y_train: (120000,)
Forma de y_test: (30000,)


In [33]:

# Entrenar y evaluar cada modelo
for name, model in models.items():
    trained_model = train_and_evaluate_model_with_weights(model, X_train, y_train, X_test, y_test, monto_test, name)

C:\Users\Ubits.PC220318\AppData\Roaming\Python\Python313\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


---------------------------------------------------

--- Evaluando modelo: Regresión Logística ---
Matriz de confusión:
[[22184  6316]
 [  375  1125]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.98      0.78      0.87     28500
           1       0.15      0.75      0.25      1500

    accuracy                           0.78     30000
   macro avg       0.57      0.76      0.56     30000
weighted avg       0.94      0.78      0.84     30000

AUC-ROC: 0.8512

Ganancia estimada para Regresión Logística: 190604.58
---------------------------------------------------

--- Evaluando modelo: Random Forest ---
Matriz de confusión:
[[28434    66]
 [ 1262   238]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.96      1.00      0.98     28500
           1       0.78      0.16      0.26      1500

    accuracy                           0.96     30000
   macro avg       0.87      0.58 

C:\Users\Ubits.PC220318\AppData\Roaming\Python\Python313\site-packages\xgboost\core.py:158: UserWarning: [11:20:41] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


---------------------------------------------------

--- Evaluando modelo: XGBoost ---
Matriz de confusión:
[[25331  3169]
 [  467  1033]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.98      0.89      0.93     28500
           1       0.25      0.69      0.36      1500

    accuracy                           0.88     30000
   macro avg       0.61      0.79      0.65     30000
weighted avg       0.95      0.88      0.90     30000

AUC-ROC: 0.8831

Ganancia estimada para XGBoost: 230766.66


## Prueba con los datos balanceados mediante SMOTE + Tomek Links

In [34]:
df_processed_balanced = pd.read_csv('../data/df_processed_balanced.csv')

# Separar características (X) y variable objetivo (y) del dataset balanceado
X_balanced = df_processed_balanced.drop(columns=['fraude','monto'])
y_balanced = df_processed_balanced['fraude']
monto_balanced = df_processed_balanced['monto']  # Para calcular la ganancia

# Dividir los datos balanceados en conjuntos de entrenamiento y prueba
X_train_balanced, X_test_balanced, y_train_balanced, y_test_balanced, monto_train, monto_test = train_test_split(
    X_balanced, y_balanced, monto_balanced, test_size=0.2, random_state=42, stratify=y_balanced
)

# Verificar las formas de los conjuntos
print(f"Forma de X_train_balanced: {X_train_balanced.shape}")
print(f"Forma de X_test_balanced: {X_test_balanced.shape}")
print(f"Forma de y_train_balanced: {y_train_balanced.shape}")
print(f"Forma de y_test_balanced: {y_test_balanced.shape}")

# Definir los modelos
models = {
    "Regresión Logística": LogisticRegression(random_state=42, max_iter=1000),
    "Random Forest": RandomForestClassifier(random_state=42, n_estimators=100),
    "XGBoost": XGBClassifier(random_state=42, use_label_encoder=False, eval_metric='logloss')
}

Forma de X_train_balanced: (227774, 71)
Forma de X_test_balanced: (56944, 71)
Forma de y_train_balanced: (227774,)
Forma de y_test_balanced: (56944,)


In [35]:
# Entrenar y evaluar cada modelo con el conjunto balanceado
for name, model in models.items():
    trained_model = train_and_evaluate_model(model, X_train_balanced, y_train_balanced, X_test_balanced, y_test_balanced, monto_test, name)

C:\Users\Ubits.PC220318\AppData\Roaming\Python\Python313\site-packages\sklearn\linear_model\_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(



--- Evaluando modelo: Regresión Logística ---
Matriz de confusión:
[[24602  3870]
 [ 4966 23506]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.83      0.86      0.85     28472
           1       0.86      0.83      0.84     28472

    accuracy                           0.84     56944
   macro avg       0.85      0.84      0.84     56944
weighted avg       0.85      0.84      0.84     56944

AUC-ROC: 0.9248

Ganancia estimada para Regresión Logística: 75016.18

--- Evaluando modelo: Random Forest ---
Matriz de confusión:
[[27574   898]
 [ 1171 27301]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.96      0.97      0.96     28472
           1       0.97      0.96      0.96     28472

    accuracy                           0.96     56944
   macro avg       0.96      0.96      0.96     56944
weighted avg       0.96      0.96      0.96     56944

AUC-ROC: 0.9934

Ganancia esti

C:\Users\Ubits.PC220318\AppData\Roaming\Python\Python313\site-packages\xgboost\core.py:158: UserWarning: [11:21:56] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)



--- Evaluando modelo: XGBoost ---
Matriz de confusión:
[[27758   714]
 [ 2054 26418]]

Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.93      0.97      0.95     28472
           1       0.97      0.93      0.95     28472

    accuracy                           0.95     56944
   macro avg       0.95      0.95      0.95     56944
weighted avg       0.95      0.95      0.95     56944

AUC-ROC: 0.9869

Ganancia estimada para XGBoost: 199708.09
